In [30]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from scipy.ndimage import filters
import scipy.misc
import time

In [31]:
class ProcessImage:
    
    k = np.sqrt(2)  # Gaussian blur factor
    constFactor = np.sqrt(k**2 - 1)
    sigma = 1.6
    s = 5           # number of images per octave
    numOctave = 4   # number of octaves
    
    def __init__(self, arr):
        """Constructor: pass an array"""
        self.imgArr = np.array(arr)
        self.imgOrigin = np.array(arr)   # store a copy of original image
    
    def plot(self, ti=""):
        plt.imshow(self.imgArr, cmap='Greys_r')
        plt.title(ti)
    
    def saveIm(self, saveName):
        self.imgOrigin.save(saveName)
        
    def getSize(self):
        return self.imgArr.shape
        
    def gaussianBlur(self, sig):
        # sig is the standard deviation of the gaussian kernel
        return filters.gaussian_filter(self.imgArr, sig)

    def halfSize(self):
        """Take every second pixels in both rows and columns."""
        return self.imgArr[::2, ::2]

In [32]:
def doubleImage(X):
    """Use linear interpolation on all pixels for preprocessing original image,
       preventing from discarding useful information.
       Input: an image array. """
    
    m, n = X.shape
    XX = np.zeros((2*m-1,2*n-1))
    # fill X in every second pixel in XX
    for i in range(m):
        for j in range(n):
            XX[2*i,2*j] = X[i,j]
            
    # then take the mean
    for i in range(1, 2*m-1, 2):
        XX[i,:] = (XX[i-1,:] + XX[i+1,:]) / 2.
    for j in range(1, 2*n-1, 2):
        XX[:,j] = (XX[:,j-1] + XX[:,j+1]) / 2.
    return XX

In [39]:
def scaleSpace(imName, showPlot=False):
    """Computes all scale spaces in all octaves."""
    
    # 1. load original image
    origin = np.array(Image.open(imName).convert('L'))
    im_pre = ProcessImage(origin)
#     im_pre.plot("im_pre: %d * %d" % im_pre.getSize())
#     plt.show()
    
    # double the size
    im_pre = ProcessImage(im_pre.gaussianBlur(0.5));    # initial blur at least sigma = 0.5
    im = ProcessImage(doubleImage(im_pre.imgArr))
#     im.plot("Original: %d * %d" % im.getSize())
#     plt.show()
    
    # 2. creating a list with all gaussian blur images with original resolution
    scaleList = []
    # octave 1: \sigma --> k^4\sigma
    for i in range(ProcessImage.s):
        newArr = im.gaussianBlur(ProcessImage.sigma * ProcessImage.k**i)
        scaleList.append(ProcessImage(newArr))

    # octave 2:  k^2\sigma --> k^6*\sigma, 1/2 size
    # octave 3:  k^4\sigma --> k^8*\sigma, 1/4 size
    # octave 4:  k^6\sigma --> k^10*\sigma, 1/8 size
    factor = 4
    for j in range(3):
        for i in range(3):
            scaleList.append(ProcessImage(scaleList[-3].halfSize()))       # the image to halfSize is always at index -3
        for i in range(factor, factor+2):
            newSig = ProcessImage.k**i * ProcessImage.constFactor * ProcessImage.sigma
            newArr = scaleList[-1].gaussianBlur(newSig)
            scaleList.append(ProcessImage(newArr))
        factor += 2


    # test for sizes
#     print([scaleList[j].getSize() for j in range(len(scaleList))])

    # 3. plot all octaves
    if showPlot == True:
        axarr = np.zeros((ProcessImage.numOctave, ProcessImage.s))
        for row in range(ProcessImage.numOctave):
            fig, axarr = plt.subplots(1, ProcessImage.s, sharey=True)
            plt.suptitle('Octave %d' % (row+1))
            for col in range(len(scaleList[:ProcessImage.s])):
                axarr[col].imshow(scaleList[row*ProcessImage.s+col].imgArr, cmap='Greys_r')
                currSigma = ProcessImage.k ** (2*row+col) * ProcessImage.sigma
                axarr[col].set_title('$\sigma = %0.4f$' % currSigma)
        plt.show()
        
    return (scaleList, im)

In [34]:
def DoG_generator(scaleList):
    """Returning list of numpy arrays."""
    DoG = []
    for oc in range(ProcessImage.numOctave):
        for i in range(ProcessImage.s - 1):
            DoG.append(scaleList[i+1+oc*ProcessImage.s].imgArr - scaleList[i+oc*ProcessImage.s].imgArr)
    return DoG

In [35]:
def localExtremaDetection(DoG):
    """Detect the local extrema of D(x,y,\sigma)."""
    extrema = []
    for oc in range(ProcessImage.numOctave):
#     for oc in range(1,2):
        m, n = DoG[oc*(ProcessImage.s-1)].shape
        for i in range(1,3):    # scale level number
            for x in range(1,m):
                for y in range(1,n):
                    # for a given sample point, compare it with 26 neighbors
                    flag = True
                    neighbors = np.array([DoG[j+oc*(ProcessImage.s-1)][x-1:x+2,y-1:y+2] for j in range(i-1,i+2)]).ravel()
                    neighbors = np.delete(neighbors, len(neighbors)/2)
                    sign = np.sign(neighbors[0] - DoG[i+oc*(ProcessImage.s-1)][x,y])
                    for num in neighbors[1:]:
                        if (np.sign(num - DoG[i+oc*(ProcessImage.s-1)][x,y]) != sign):
                            flag = False
                            break
                    # if not break, (x,y) is an extrema
                    if flag:
                        # map the point by a multiplier for different scales
                        extrema.append((2**oc*x, 2**oc*y))
                
    return extrema

In [58]:
def showKeypoints(extrema, im):
    """Input extrema is a list of pixel coordinates; im is a ProcessImage instance."""
    extrema = list(set(extrema))
    extrema = np.asarray(extrema)
    im.plot("Size: %dpx * %dpx, number of keypoints = %d" % (im.getSize()[0], im.getSize()[1], len(extrema)))
    plt.plot(extrema[:,1], extrema[:,0],'b.', label='keypoints')   # notice the order of indices
    plt.legend(loc = 'upper left')
    plt.show()

In [45]:
def main():
    # creating LoG and DoG
    (scaleList, im) = scaleSpace('house.jpg', showPlot=False)
    DoG = DoG_generator(scaleList)

    # detecting local extrema
    t0 = time.clock()
    extrema = localExtremaDetection(DoG)
    print("Time spent detecting local extrema: %0.2f" % (time.clock()-t0))

    # plot keypoints on original graph
    showKeypoints(extrema, im)

Time spent detecting local extrema: 5.79


In [44]:
if __name__ == "__main__":
    main()

Time spent detecting local extrema: 5.72


TypeError: %d format: a number is required, not tuple